In [1]:
import pyspark
from pyspark.sql import SparkSession
# import pandas as pd
from pyspark.sql import types
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

22/02/28 10:08:18 WARN Utils: Your hostname, shura-tower resolves to a loopback address: 127.0.1.1; using 10.0.1.7 instead (on interface wlo1)
22/02/28 10:08:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/02/28 10:08:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df = spark.read.parquet('fhvhv/2021/02/')

In [5]:
df.registerTempTable('fhvhv_data')

##### Question 3. Count records

How many taxi trips were there on February 15?

Consider only trips that started on February 15

In [34]:
spark.sql("""
SELECT
    count(1)
FROM
    fhvhv_data
WHERE
    DATE(pickup_datetime) = '2021-02-15' 
""").show()

+--------+
|count(1)|
+--------+
|  367170|
+--------+



##### Question 4. Longest trip for each day

Now calculate the duration for each trip.

Trip starting on which day was the longest?

In [10]:
spark.sql("""
SELECT
    DATE(pickup_datetime) AS date,
    (unix_timestamp(dropoff_datetime) - unix_timestamp(pickup_datetime)) AS duration
FROM
    fhvhv_data
ORDER BY
    duration DESC
LIMIT
    10
""").show()

+----------+--------+
|      date|duration|
+----------+--------+
|2021-02-11|   75540|
|2021-02-17|   57221|
|2021-02-20|   44039|
|2021-02-03|   40653|
|2021-02-19|   37577|
|2021-02-25|   35010|
|2021-02-20|   34806|
|2021-02-18|   34612|
|2021-02-18|   34555|
|2021-02-10|   34169|
+----------+--------+



##### Question 5. Most frequent dispatching_base_num

Now find the most frequently occurring dispatching_base_num in this dataset.

How many stages this spark job has?

(Note: the answer may depend on how you write the query, so there are multiple correct answers. Select the one you have.)

In [43]:
spark.sql("""
SELECT
    dispatching_base_num, COUNT(1)
FROM
    fhvhv_data
GROUP BY
    dispatching_base_num
""").show()

+--------------------+--------+
|dispatching_base_num|count(1)|
+--------------------+--------+
|              B02876|  215693|
|              B03136|    1741|
|              B02877|  198938|
|              B02869|  429720|
|              B02883|  251617|
|              B02835|  189031|
|              B02884|  244963|
|              B02880|  115716|
|              B02878|  305185|
|              B02836|  128978|
|              B02872|  882689|
|              B02512|   41043|
|              B02867|  200530|
|              B02866|  311089|
|              B02871|  312364|
|              B02889|  138762|
|              B02844|    3502|
|              B02510| 3233664|
|              B02888|  169167|
|              B02682|  303255|
+--------------------+--------+
only showing top 20 rows



##### Question 6. Most common locations pair

Find the most common pickup-dropoff pair.

For example:

"Jamaica Bay / Clinton East"

Enter two zone names separated by a slash

If any of the zone names are unknown (missing), use "Unknown". For example, "Unknown / Clinton East".

In [45]:
!wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv

--2022-02-23 11:51:05--  https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.199.232
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.199.232|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12322 (12K) [application/octet-stream]
Saving to: ‘taxi+_zone_lookup.csv’

taxi+_zone_lookup.c 100%[===================>]  12.03K  --.-KB/s    in 0.002s  

2022-02-23 11:51:06 (5.86 MB/s) - ‘taxi+_zone_lookup.csv’ saved [12322/12322]



In [46]:
zone_df = spark.read \
    .option("header", "true") \
    .csv('taxi+_zone_lookup.csv')

zone_df.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [47]:
schema_zone = types.StructType([
    types.StructField('LocationID', types.IntegerType(), True),
    types.StructField('Borough', types.StringType(), True),
    types.StructField('Zone', types.StringType(), True),
    types.StructField('service_zone', types.StringType(), True)
])

In [48]:
zone_df = spark.read \
    .option("header", "true") \
    .schema(schema_zone) \
    .csv('taxi+_zone_lookup.csv')

In [51]:
zone_df.registerTempTable('zones')

In [56]:
spark.sql("""
SELECT
    COALESCE(pu_zone.Zone, 'Unknown') AS pickup_zone,
    COALESCE(do_zone.Zone, 'Unknown') AS dropoff_zone
FROM
    fhvhv_data AS fhv
INNER JOIN
    zones as pu_zone
ON
    fhv.PULocationID = pu_zone.LocationID
INNER JOIN
    zones as do_zone
ON
    fhv.DOLocationID = do_zone.LocationID
""").show(10)

+--------------------+--------------------+
|         pickup_zone|        dropoff_zone|
+--------------------+--------------------+
|      Yorkville West|     Lenox Hill East|
|        Baisley Park|     Hammels/Arverne|
|Williamsbridge/Ol...|Williamsbridge/Ol...|
| Crown Heights North|          Ocean Hill|
|   East Harlem North|   East Harlem North|
|     Hammels/Arverne|        Far Rockaway|
|           Rego Park|            Woodside|
| Crown Heights North|Prospect-Lefferts...|
|Spuyten Duyvil/Ki...|Upper East Side N...|
|Marine Park/Mill ...|Flatbush/Ditmas Park|
+--------------------+--------------------+
only showing top 10 rows



In [59]:
spark.sql("""
SELECT
    CONCAT(t.pickup_zone, ' / ', t.dropoff_zone) AS pu_do_pair,
    COUNT(1) as count
FROM
    (SELECT
        COALESCE(pu_zone.Zone, 'Unknown') AS pickup_zone,
        COALESCE(do_zone.Zone, 'Unknown') AS dropoff_zone
    FROM
        fhvhv_data AS fhv
    INNER JOIN
        zones as pu_zone
    ON
        fhv.PULocationID = pu_zone.LocationID
    INNER JOIN
        zones as do_zone
    ON
        fhv.DOLocationID = do_zone.LocationID
    ) AS t
GROUP BY
    pu_do_pair
ORDER BY
    count DESC
LIMIT
    10
""").show(5, False)

+-----------------------------------------+-----+
|pu_do_pair                               |count|
+-----------------------------------------+-----+
|East New York / East New York            |45041|
|Borough Park / Borough Park              |37329|
|Canarsie / Canarsie                      |28026|
|Crown Heights North / Crown Heights North|25976|
|Bay Ridge / Bay Ridge                    |17934|
+-----------------------------------------+-----+
only showing top 5 rows

